# SQL Server Database

In [1]:
import sqlalchemy
from sqlalchemy import text
from functools import partial
import pandas as pd
from sql_connector import Database

## Create Engine

In [4]:
nba = Database(database='NBA')

## Create Table

### Columns to Create

In [3]:
player_shotLocations = pd.read_csv(r"D:\lianz\Desktop\Python\personal_projects\nba\data\player_shotLocations.csv", header=[0,1], skipinitialspace=True,)
games_played = pd.read_csv(r"D:\lianz\Desktop\Python\personal_projects\nba\data\games_played.csv", parse_dates=['DateTime'])

### Execute SQL to Create Table

In [ ]:
create_team_table = text("""
CREATE TABLE teams(
team_id INT PRIMARY KEY NOT NULL,
team_name VARCHAR(20),
team_abbreviation CHAR(3)
)
""")

create_player_table = text("""
CREATE TABLE players(
player_id INT PRIMARY KEY,
player_name VARCHAR(30)
)
""")

create_team_arena = text("""
CREATE TABLE arena_history(
arena_id INT PRIMARY KEY,
arena_name VARCHAR(30),
team_year INT,
team_id INT FOREIGN KEY REFERENCES teams(team_id)
)
""")

create_player_team_table = text("""
CREATE TABLE player_team_history(
id INT NOT NULL IDENTITY(1,1),
player_id INT FOREIGN KEY REFERENCES players(player_id),
player_age INT,
team_id INT FOREIGN KEY REFERENCES teams(team_id),
year VARCHAR(10)
)
""")

create_games_played = text("""
CREATE TABLE games_played(
game_id CHAR(18) PRIMARY KEY,
game_date datetime,
visitor_team_id INT FOREIGN KEY REFERENCES teams(team_id), 
visitor_pts INT NOT NULL,
home_team_id INT FOREIGN KEY REFERENCES teams(team_id),
home_pts INT NOT NULL,
overtime VARCHAR(5),
attendance INT,
arena_id INT
)
""")

with nba.engine.begin() as conn:
    conn.execute(create_team_table)
    conn.execute(create_player_table)
    conn.execute(create_team_arena)
    conn.execute(create_player_team_table)
    conn.execute(create_games_played)

## Add Columns to Table

In [ ]:
add_arena_col = text("""
ALTER TABLE teams
ADD arena_name VARCHAR(30);
""")

with nba.engine.begin() as conn:
    conn.execute(add_arena_col)

## Insert Table

In [ ]:
games_played['game_id'].apply(lambda x: len(x)).value_counts()

In [ ]:
teams_df = games_played.rename(columns={'Home':'team_name', 'Home_short':'team_abbreviation',})[['DateTime','team_name','team_abbreviation','Arena']].copy()
teams_df['year'] = teams_df['DateTime'].apply(lambda x: str(x.year))
teams_df.drop(columns=['DateTime'], inplace=True)
teams_df = teams_df.dropna(subset=['Arena']).drop_duplicates()
teams_df['team_id'] = teams_df['team_abbreviation'] + teams_df['year'] + 

### Execute SQL Statement to INSERT

In [ ]:
insert_table = text("""
INSERT INTO teams (team_id, team_name)
VALUES (:player_id, :player_name)
""")

with nba.engine.begin() as conn:
    conn.execute(insert_table, [{"player_id": 1, "player_name": 1}, {"player_id": 2, "player_name": 4}],)

# Drop Table

In [ ]:
drop_table = text("""
DROP TABLE player_team_history;
DROP TABLE games_played;
DROP TABLE arena_history;
DROP TABLE players;
DROP TABLE teams;

""")

with nba.engine.begin() as conn:
    conn.execute(drop_table)

# Drop Column from Table

In [ ]:
column_to_drop = '[index]'
table_name = 'play_by_play'
drop_column = text(f"""
ALTER TABLE {table_name}
DROP COLUMN {column_to_drop}
""")

with nba.engine.begin() as conn:
    conn.execute(drop_column)
    print(f'Column {column_to_drop} has been dropped from table {table_name}')

# Access Data from Table

In [5]:
with nba.engine.connect() as conn:
    pbp = conn.execute(text("SELECT * FROM play_by_play WHERE game_id = '202304301530GSWSAC'"))
    rows = pbp.fetchall()

pbp_df = pd.DataFrame(rows, columns=pbp.keys())

2023-05-09 22:09:39,413 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2023-05-09 22:09:39,413 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-09 22:09:39,415 INFO sqlalchemy.engine.Engine SELECT schema_name()
2023-05-09 22:09:39,415 INFO sqlalchemy.engine.Engine [generated in 0.00058s] ()
2023-05-09 22:09:39,426 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2023-05-09 22:09:39,426 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ()
2023-05-09 22:09:39,428 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2023-05-09 22:09:39,429 INFO sqlalchemy.engine.Engine [generated in 0.00039s] ()
2023-05-09 22:09:39,432 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-09 22:09:39,433 INFO sqlalchemy.engine.Engine SELECT * FROM play_by_play WHERE game_id = '202304301530GSWSAC'
2023-05-09 22:09:39,433 INFO sqlalchemy.engine.Engine [ge